In [0]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 5.0MB/s 
     |████████████████████████████████| 2.7MB 42.3MB/s 


In [11]:
from re import split
from networkx import Graph
from networkx import pagerank
from itertools import combinations
from collections import Counter
from konlpy.tag import Okt


class Sentence(object):
    okt = Okt()

    def __init__(self, text, index=0):
        self.index = index
        self.text = text.strip()
        self.tokens = self.okt.phrases(self.text)
        self.bow = Counter(self.tokens)

    def __str__(self):
        return self.text

    def __hash__(self):
        return self.index


class TextRank(object):
    def __init__(self, text):
        self.text = text.strip()
        self.build()

    def build(self):
        self._build_sentences()
        self._build_graph()
        self.pageranks = pagerank(self.graph, weight='weight')
        self.reordered = sorted(self.pageranks, key=self.pageranks.get, reverse=True)

    def _build_sentences(self):
        dup = {}
        candidates = split(r'(?:(?<=[^0-9])\.|\n)', self.text)
        self.sentences = []
        index = 0
        for candidate in candidates:
            while len(candidate) and (candidate[-1] == '.' or candidate[-1] == ' '):
                candidate = candidate.strip(' ').strip('.')
            if len(candidate) and candidate not in dup:
                dup[candidate] = True
                self.sentences.append(Sentence(candidate + '.', index))
                index += 1
        del dup
        del candidates

    def _build_graph(self):
        self.graph = Graph()
        self.graph.add_nodes_from(self.sentences)
        for sent1, sent2 in combinations(self.sentences, 2):
            weight = self._jaccard(sent1, sent2)
            if weight:
                self.graph.add_edge(sent1, sent2, weight=weight)

    def _jaccard(self, sent1, sent2):
        p = sum((sent1.bow & sent2.bow).values())
        q = sum((sent1.bow | sent2.bow).values())
        return p / q if q else 0

    def summarize(self, count=3, verbose=True):
        results = sorted(self.reordered[:count], key=lambda sentence: sentence.index)
        results = [result.text for result in results]
        if verbose:
            return '\n'.join(results)
        else:
            return results

textfull = '접수. 서울시 청년수당 지원 대상이 되어서 지원을 하려고 합니다. 근데 내일배움카드 훈련장려금 대상자는 대상제외로 되어있더군요. 재직자때 발급 받은 내일배움 카드가 있고 2017년도에 수업을 받았었고 장려금도 남아있으며, 유효기간이 내년 12월까지로 되어있습니다. 2월 28일부터 퇴사한 후 내일배움카드로 수업을 들으려고 신청해 놓은것이 있습니다. 저는 참여 수당 지원 대상에서 제외 되나요?. 아님 애초에 저는 이미 재직자 내일배움 카드가 있으니 서울시 청년수당 참여 대상이 아닌건가요? 애초에 참여 대상자가 아니면, 저는 내일배움카드로 수업을 받으려고 합니다.'

textrank = TextRank(textfull)

print(type(textrank.summarize(3, verbose=False)))
print(textrank.summarize(3, verbose=False))  # up to 3 sentences, returned as list

<class 'list'>
['서울시 청년수당 지원 대상이 되어서 지원을 하려고 합니다.', '근데 내일배움카드 훈련장려금 대상자는 대상제외로 되어있더군요.', '아님 애초에 저는 이미 재직자 내일배움 카드가 있으니 서울시 청년수당 참여 대상이 아닌건가요? 애초에 참여 대상자가 아니면, 저는 내일배움카드로 수업을 받으려고 합니다.']
